In [2]:
import pandas as pd
import seaborn as sns

from pathlib import Path
import sys

src_path = Path.cwd().parent / "src"
if src_path not in sys.path:
    sys.path.append(str(src_path))

# Loading a dataset

In [3]:
summary_data = pd.read_feather("/mnt/upramdya_data/MD/Ballpushing_Exploration/Datasets/250403_BallpushingMetrics_Data/summary/250403_pooled_summary.feather")

summary_data.head()

,index,nb_events,max_event,max_event_time,max_distance,final_event,final_event_time,nb_significant_events,significant_ratio,first_significant_event,...,Genotype,Period,FeedingState,Orientation,Light,Crossing,BallType,Used_to,Magnet,Peak
0,fly_0_ball_0,27.314579,27.0,1184.758621,95.360159,None,None,8.0,0.081633,17.0,...,,,,,,,,,,NaN
1,fly_0_ball_0,1.389688,NaN,NaN,41.298635,None,None,0.0,0.000000,NaN,...,,,,,,,,,,NaN
2,fly_0_ball_0,14.224436,23.0,1295.793103,115.381141,None,None,5.0,0.098039,7.0,...,,,,,,,,,,NaN
3,fly_0_ball_0,1.955383,0.0,9.034483,14.427183,None,None,1.0,0.142857,0.0,...,,,,,,,,,,NaN
4,fly_0_ball_0,8.363857,15.0,2714.793103,18.319380,None,None,1.0,0.033333,18.0,...,,,,,,,,,,NaN


Check some overarching statistics about the data

In [5]:
# Compute how many unique fly IDs are in each group

len(summary_data["fly"].unique())

430